In [392]:
import re
import os
import nltk
import ftfy
import ast
import json
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords

from sklearn.preprocessing import Binarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix




%matplotlib inline

pd.options.display.max_rows = 420

### Data Collection
- Musixmatch API
- Script used is scraper.py

In [310]:
json_data = ""
json_data2 = ""
json_data3 = ""
json_data4 = ""

def read_file(filepath):
    data = ""
    with open(filepath) as f:
        data = f.readlines()
    return data

json_data = read_file("rnb:hiphop1.txt")
json_data2 = read_file("rnb:hiphop2.txt")
json_data3 = read_file("rnb:hiphop3.txt")
json_data4 = read_file("rnb:hiphop4.txt")

data = [ast.literal_eval(item) for item in json_data]
data2 = [ast.literal_eval(item) for item in json_data2]
data3 = [ast.literal_eval(item) for item in json_data3]
data4 = [ast.literal_eval(item) for item in json_data4]

In [311]:
raw_data = data[0]
raw_data2 = data2[0]
raw_data3 = data3[0]
raw_data4 = data4[0]

df = pd.DataFrame(columns={"year", "artist", "albums"})

In [312]:
row_list = []

def add_rows(data):
    rows = []
    for i in range(len(data)):
        row = {}
        row['year'] = data[i]['year']
        row['artist'] = data[i]['artist']
        row['albums'] = data[i]['albums']
        rows.append(row)
    return rows

row_list.extend(add_rows(raw_data))
row_list.extend(add_rows(raw_data2))
row_list.extend(add_rows(raw_data3))
row_list.extend(add_rows(raw_data4))

df = pd.DataFrame(row_list)
df

,albums,artist,year
0,"{('King of Pop: The Indian Collection', 109682...",Michael Jackson,1980
1,"{('The Very Best Of', 18953579): {'Friends': '...",Shalamar,1980
2,{},"Ray, Goodman & Brown",1980
3,"{('Planets of Life', 10919184): {'Seems Like I...",The Whispers,1980
4,"{('Ain't We Funkin' Now', 13885304): {'I'll Be...",The Brothers Johnson,1980
5,"{('Discover Further: The Isley Brothers', 1579...",The Isley Brothers,1980
6,"{('You Said', 21359468): {'You Said, You Said'...",Jermaine Jackson,1980
7,"{('No Lies', 15680166): {'No Lies': 'Wake up. ...",The S.O.S. Band,1980
8,"{('Just Be My Lady', 17117936): {'Just Be My L...",Larry Graham,1980
9,"{('Anthology', 15769731): {'Reach Out and Touc...",Diana Ross,1980


In [313]:
len(df)

465

In [314]:
df_unique_artists = df.drop_duplicates(subset=['artist'])

In [315]:
len(df_unique_artists)

344

In [316]:
temp = []
feat_re = "[&,+]"
for index, row in df_unique_artists.iterrows():
    if re.search(feat_re, row['artist']) or "Featuring" in row['artist']:
        print(row['artist'])
        continue
    hold = []
    for album in row['albums']:
        track = row["albums"][album]
        hold.append(track)
    temp.append({row['artist']: hold})

Ray, Goodman & Brown
Kool & The Gang
Yarbrough & Peoples
Ray Parker Jr. & Raydio
Diana Ross & Lionel Richie
Earth, Wind & Fire
Dreamgirls Featuring Jennifer Holliday
George Clinton Featuring Coolio
Ashford & Simpson
Maze Featuring Frankie Beverly
Post Malone Featuring Ty Dolla $ign
Post Malone Featuring 21 Savage
Bruno Mars & Cardi B
DJ Khaled Featuring Rihanna & Bryson Tiller
DJ Khaled Featuring Justin Bieber, Quavo, Chance The Rapper & Lil Wayne
Migos Featuring Lil Uzi Vert
Rae Sremmurd Featuring Gucci Mane
The Weeknd Featuring Daft Punk
D.R.A.M. Featuring Lil Yachty
Drake Featuring WizKid & Kyla
Rihanna Featuring Drake
Wiz Khalifa Featuring Charlie Puth
Rihanna & Kanye West & Paul McCartney
Usher Featuring Juicy J
Big Sean Featuring E-40
Nicki Minaj Featuring Drake, Lil Wayne & Chris Brown
Iggy Azalea Featuring Rita Ora
Iggy Azalea Featuring Charli XCX
Beyonce Featuring Jay Z
Eminem Featuring Rihanna
Drake Featuring Majid Jordan
Robin Thicke Featuring T.I. + Pharrell
Macklemore & Ry

In [317]:
len(temp)

229

In [318]:
# raw_data[1]

In [348]:
temp2 = []
for item in temp:
    artist = ""
    all_lyrics = ""
    for key, value in item.items():
        artist = key
        for song in value:
            for title, lyrics in song.items():
                lyrics = lyrics + "\n"
                all_lyrics += lyrics

    temp2.append({artist:all_lyrics})

In [374]:
nextline_re = re.compile(r"\n")
temp3 = []
for item in temp:
    each_song = []
    artist = ""
    for key, value in item.items():
        artist = key
        for song in value:
            for title, album_tracks in song.items():
                tracks = nextline_re.sub(" ", album_tracks)
                each_song.append(tracks)
    temp3.append({artist:each_song})

In [321]:
# len(temp2)

In [322]:
#this writes the combined lyrics for songs at once
for i in temp2:
    for artist, lyrics in i.items():
        filename = "/Users/oyinlola/Desktop/MSc Data Science/LENT TERM/SCC 413 - Applied Data Mining/Research Projects/Lyrics/" + artist + ".txt"
        with open(filename, "w") as text_file:
            text_file.write(lyrics)
        print("Saved ", artist)

Saved  Michael Jackson
Saved  Shalamar
Saved  The Whispers
Saved  The Brothers Johnson
Saved  The Isley Brothers
Saved  Jermaine Jackson
Saved  The S.O.S. Band
Saved  Larry Graham
Saved  Diana Ross
Saved  George Benson
Saved  Tom Browne
Saved  Stevie Wonder
Saved  Lakeside
Saved  The Gap Band
Saved  Smokey Robinson
Saved  A Taste Of Honey
Saved  Chaka Khan
Saved  Rick James
Saved  Frankie Smith
Saved  Evelyn King
Saved  Four Tops
Saved  Luther Vandross
Saved  Roger
Saved  Daryl Hall John Oates
Saved  Skyy
Saved  Richard "Dimples" Fields
Saved  Deniece Williams
Saved  Dazz Band
Saved  Zapp
Saved  Aretha Franklin
Saved  Marvin Gaye
Saved  Michael Jackson And Paul McCartney
Saved  New Edition
Saved  Gladys Knight And The Pips
Saved  Mtume
Saved  Donna Summer
Saved  Lionel Richie
Saved  Debarge
Saved  Patti LaBelle
Saved  Cheryl Lynn
Saved  Rockwell
Saved  Cameo
Saved  O'Bryan
Saved  Prince
Saved  Ray Parker Jr.
Saved  Billy Ocean
Saved  Prince And The Revolution
Saved  Midnight Star
Saved

In [386]:
#write song to files: a song on a line 
for i in temp3:
    for artist, all_tracks in i.items():
        filename = "/Users/oyinlola/Desktop/MSc Data Science/LENT TERM/SCC 413 - Applied Data Mining/Research Projects/Lyrics/new/" + artist + ".txt"
        with open(filename, "w") as text_file:
            for track in all_tracks:
                track = track + "\n"
                text_file.write(track)
        print("Saved ", artist)

Saved  Michael Jackson
Saved  Shalamar
Saved  The Whispers
Saved  The Brothers Johnson
Saved  The Isley Brothers
Saved  Jermaine Jackson
Saved  The S.O.S. Band
Saved  Larry Graham
Saved  Diana Ross
Saved  George Benson
Saved  Tom Browne
Saved  Stevie Wonder
Saved  Lakeside
Saved  The Gap Band
Saved  Smokey Robinson
Saved  A Taste Of Honey
Saved  Chaka Khan
Saved  Rick James
Saved  Frankie Smith
Saved  Evelyn King
Saved  Four Tops
Saved  Luther Vandross
Saved  Roger
Saved  Daryl Hall John Oates
Saved  Skyy
Saved  Richard "Dimples" Fields
Saved  Deniece Williams
Saved  Dazz Band
Saved  Zapp
Saved  Aretha Franklin
Saved  Marvin Gaye
Saved  Michael Jackson And Paul McCartney
Saved  New Edition
Saved  Gladys Knight And The Pips
Saved  Mtume
Saved  Donna Summer
Saved  Lionel Richie
Saved  Debarge
Saved  Patti LaBelle
Saved  Cheryl Lynn
Saved  Rockwell
Saved  Cameo
Saved  O'Bryan
Saved  Prince
Saved  Ray Parker Jr.
Saved  Billy Ocean
Saved  Prince And The Revolution
Saved  Midnight Star
Saved

### Data Preprocessing

In [499]:
sections_re = re.compile(r"(\[)?\[(.*?)\](\])?") #anything in square bracket
par_re = re.compile(r"\((.*?)\)") #anything in parentheses
verse_re = re.compile(r"([0-9a-z ]+)?verse(:)?[\s](\w.)?") #verse headers
repeat_re = re.compile(r"(\n)?repeat[a-z .\(\)\d]+") #repeat headers
chorus_re = re.compile(r"chorus([a-z .\(\)\d:]+)?") #chorus headers
hook_re = re.compile(r"(\n)?hook[:| |\n]([a-z .\(\)\d]+)?") #hook headers
title_re = re.compile(r"(\n)?title:([\w+ ]+)?") #title headers
singer_re = re.compile(r"([a-z &]+)?\w+:") #name of singer indicating who is singing currently especially for featured artists 
punt_re = re.compile(r"[^a-zA-Z'&\n ]+") #irrelevant puctuations
common_re = re.compile(r"******* This Lyrics is NOT for Commercial use *******")

# all_words = re.sub(r'[\(\[].*?[\)\]]', '', all_words) ---- check this out

tokenise_re = re.compile(r"(\[[^\]]+\]|[-'\w]+|[^\s\w\[']+)") #([]|words|other non-space)

def preprocess(text):
    text = text.lower()
    p_text = sections_re.sub("", text)
    p_text = par_re.sub("", p_text)
    p_text = verse_re.sub("", p_text)
    p_text = repeat_re.sub("", p_text)
    p_text = chorus_re.sub("", p_text)
    p_text = hook_re.sub("", p_text)
    p_text = title_re.sub("", p_text)
    p_text = singer_re.sub("", p_text)
    p_text = punt_re.sub("", p_text)
    p_text = common_re.sub("", p_text)
    p_text = ftfy.fix_text(p_text)
    return p_text.lower()  #return lyrics in the same case

def replace_apostrophe(lyrics):
    for text in lyrics:
        if "'" in text:
            shortened = text.split("'")
            if shortened[1] == "ll":
                text = shortened[0] + " will"
            elif shortened[1] == "m":
                text = shortened[0] + " am"
            elif shortened[1] == "ve":
                text = shortened[0] + " have"
            elif shortened[1] == "re":
                text = shortened[0] + " are"
            elif shortened[1] == "cause":
                text = "because"
            elif shortened[1] == "":
                text = shortened[0] + "g"
            elif text == "y'all":
                text = "you all"
            elif text == "'em":
                text = "them"
            elif text == "'lone":
                text = "alone"
            elif text == "c'mon":
                text = "common"
            elif shortened[1] == "t" and shortened != "ain":
                text = shortened[0] + " not"
    return lyrics
          
     
def tokenise(text):
    return tokenise_re.findall(text)

nltk.download('stopwords')
nltk_stopwords = set(stopwords.words('english'))

def stopwords_file(file):
    with open(file) as f:
        items = []
        lines = f.readlines()
        for line in lines:
            items.append(line.strip())
    return items

stanford_stopwords = stopwords_file("stanford_stopwords.txt")

def remove_stopwords(tokens):
    nltk_filtered = [word for word in tokens if word not in nltk_stopwords]
#     filtered = [word for word in nltk_filtered if word not in stanford_stopwords]
    return nltk_filtered

def frequency_analysis(tokens):
    freq = nltk.FreqDist(tokens)
    for key,val in freq.most_common(20):
        print(key,val,sep="\t")
    freq.plot(20, cumulative=False)

error: nothing to repeat at position 0

In [500]:
class Artist:
    def __init__(self, obj):
        self.gender = obj['gender']
        self.lyrics = ""
        self.tokens = []
        self.pos_list = []
        self.num_tokens = 0
        self.pos_fql = Counter()
        self.bigrams = Counter()
        self.trigrams = Counter()
        self.tokens_fql = Counter()
        
    
    tokenise_re = re.compile(r"(\[[^\]]+\]|[-'\w]+|[^\s\w\[']+)") #([]|words|other non-space)
    def tokenise(self, text):
        return tokenise_re.findall(text)

    def extract_features(self, track):
        self.lyrics += track
        p_lyrics = replace_apostrophe(track)  #replace apostrophe
        p_lyrics = preprocess(p_lyrics) 
        tokens = self.tokenise(p_lyrics)
        tokens = remove_stopwords(tokens)
        self.tokens.extend(tokens)
        self.num_tokens += len(tokens)
        self.tokens_fql.update(tokens)
        pos_tagged = nltk.pos_tag(tokens)
        pos = [tag[1] for tag in pos_tagged]
        self.pos_fql.update(pos)
        self.pos_list.extend(pos)
        self.bigrams.update(self.ngrams(tokens, 2))
        self.trigrams.update(self.ngrams(tokens, 3))
        
    def ngrams(self, tokens, n, sep = "_", buffer="^"):
        buffered = [buffer] * (n-1) + tokens + [buffer] * (n-1) 
        return [sep.join(buffered[i:i+n]) for i in range(len(buffered)-n+1)]
    
    def extract_features_from_lyrics(self, tracks):
        for lyrics in tracks:
            extract_features(lyrics)

In [501]:
class DocumentProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, process_method):
        self.process_method = process_method
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, documents):
        for document in documents:
            yield self.process_method(document)

In [502]:
def get_tokens_fql(document):
    return document.tokens_fql

def get_pos_fql(document):
    return document.pos_fql

def get_bigrams(document):
    return document.bigrams

def get_trigrams(document):
    return document.trigrams

In [503]:
def import_lyrics(file, gender):
    metadata = {'gender': gender}
    with open(file) as f:
        tracks = f.readlines()
        for track in tracks:
            song = Artist(metadata)
            song.extract_features(track)
            yield song

In [504]:
def build_corpus(folder, gender):
    corpus = []
    for file in os.listdir(folder):
        file = folder + file
        words = import_lyrics(file, gender)
        corpus.extend(words)
    return corpus

In [505]:
corpus = []

female = build_corpus("Data/female_new/", "female")
male = build_corpus("Data/male_new/", "male")

corpus.extend(female)
corpus.extend(male)

corpus

 ...]

In [506]:
y = [d.gender for d in corpus]
X = corpus


In [507]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 0, stratify=y)
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

34271 8568
34271 8568


In [508]:
model = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('bow_features', Pipeline([
                ('bow_processor', DocumentProcessor(process_method = get_tokens_fql)),
                ('pos_vectorizer', DictVectorizer()),
            ])),
        ],
    )),
    ('clf', LogisticRegression()),
])

In [509]:
def print_cv_scores_summary(name, scores):
    print("{}: mean = {:.2f}%, sd = {:.2f}%, min = {:.2f}, max = {:.2f}".format(name, scores.mean()*100, scores.std()*100, scores.min()*100, scores.max()*100))

In [510]:
cv_scores = cross_validate(model, X, y, cv=StratifiedKFold(n_splits=5, random_state=0), return_train_score=False, scoring=['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'])
print_cv_scores_summary("Accuracy", cv_scores['test_accuracy'])
print_cv_scores_summary("Precision", cv_scores['test_precision_weighted'])
print_cv_scores_summary("Recall", cv_scores['test_recall_weighted'])
print_cv_scores_summary("F1", cv_scores['test_f1_weighted'])

Accuracy: mean = 64.02%, sd = 2.82%, min = 58.72, max = 66.60
Precision: mean = 63.97%, sd = 2.86%, min = 58.67, max = 66.68
Recall: mean = 64.02%, sd = 2.82%, min = 58.72, max = 66.60
F1: mean = 63.98%, sd = 2.84%, min = 58.69, max = 66.64


In [498]:
len(corpus)

42839

In [325]:
def clean_lyrics(filepath):
    base = os.path.basename(filepath)
    base = os.path.splitext(base)
    artist = (base)[0]

    all_lyrics = read_file(filepath)
    lyrics = ""
    
    for line in all_lyrics:
        clean = preprocess(line)
        lyrics = lyrics + " " + clean

    all_lyrics_cleaned = replace_apostrophe(lyrics)
    
    print("Cleaned ", artist)
    return (all_lyrics_cleaned, artist)

In [326]:
# clean_lyrics("/Users/oyinlola/Desktop/MSc Data Science/LENT TERM/SCC 413 - Applied Data Mining/Research Projects/Lyrics/50 Cent.txt")

In [327]:
#clean all files available
folder = "/Users/oyinlola/Desktop/MSc Data Science/LENT TERM/SCC 413 - Applied Data Mining/Research Projects/Lyrics/"

for file in os.listdir(folder):
    try:
        file = folder + file
        cleaned = clean_lyrics(file)
        artist = cleaned[1]
        cleaned_lyrics = cleaned[0]
        filename = "/Users/oyinlola/Desktop/MSc Data Science/LENT TERM/SCC 413 - Applied Data Mining/Research Projects/LyricsCleaned/" + artist + ".txt"
        with open(filename, "w") as text_file:
            text_file.write(cleaned_lyrics)
        print("Saved ", artist)
    except Exception as e:
        print("Exception at >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ", file)
        continue

Cleaned  Naughty By Nature
Saved  Naughty By Nature
Cleaned  The O'Jays
Saved  The O'Jays
Cleaned  Quincy Jones With Tevin Campbell
Saved  Quincy Jones With Tevin Campbell
Cleaned  Prince
Saved  Prince
Cleaned  Jeffrey Osborne
Saved  Jeffrey Osborne
Cleaned  Trey Songz
Saved  Trey Songz
Cleaned  The Isley Brothers
Saved  The Isley Brothers
Cleaned  Pebbles
Saved  Pebbles
Cleaned  Peabo Bryson
Saved  Peabo Bryson
Cleaned  Zapp
Saved  Zapp
Cleaned  Kelly Price
Saved  Kelly Price
Cleaned  Mr. Cheeks
Saved  Mr. Cheeks
Cleaned  TLC
Saved  TLC
Cleaned  Smokey Robinson
Saved  Smokey Robinson
Cleaned  Damian Dame
Saved  Damian Dame
Cleaned  Loose Ends
Saved  Loose Ends
Cleaned  Roberta Flack
Saved  Roberta Flack
Cleaned  Johnny Kemp
Saved  Johnny Kemp
Cleaned  Rockwell
Saved  Rockwell
Cleaned  Lisa Fischer
Saved  Lisa Fischer
Cleaned  Natalie Cole
Saved  Natalie Cole
Cleaned  Ginuwine
Saved  Ginuwine
Cleaned  Johnny Gill
Saved  Johnny Gill
Cleaned  Mtume
Saved  Mtume
Cleaned  Cameo
Saved  Came

Cleaned  Commodores
Saved  Commodores
Cleaned  Destiny's Child
Saved  Destiny's Child
Cleaned  Mary J. Blige
Saved  Mary J. Blige
Cleaned  Tag Team
Saved  Tag Team
Cleaned  Deborah Cox With R.L.
Saved  Deborah Cox With R.L.
Cleaned  Sade
Saved  Sade
Cleaned  DRS
Saved  DRS
Cleaned  Prince And The Revolution
Saved  Prince And The Revolution
Cleaned  Larry Graham
Saved  Larry Graham
Cleaned  Lisa Lisa And Cult Jam
Saved  Lisa Lisa And Cult Jam
Cleaned  Jermaine Jackson
Saved  Jermaine Jackson
Cleaned  Shanice
Saved  Shanice
Cleaned  Jody Watley
Saved  Jody Watley
Cleaned  Teddy Pendergrass
Saved  Teddy Pendergrass
Cleaned  Cheryl Pepsii Riley
Saved  Cheryl Pepsii Riley
Cleaned  Lil Wayne
Saved  Lil Wayne
Cleaned  Quincy Jones
Saved  Quincy Jones
Cleaned  Carl Thomas
Saved  Carl Thomas
Cleaned  Janet Jackson
Saved  Janet Jackson
Cleaned  Maxwell
Saved  Maxwell
Cleaned  New Edition
Saved  New Edition
Cleaned  Roger
Saved  Roger
Cleaned  Jagged Edge
Saved  Jagged Edge
Cleaned  Az Yet
Saved 

In [328]:
from os import listdir
from os.path import isfile, join, splitext, split

def list_files(folder):
    textfiles = [join(folder, f) for f in listdir(folder) if isfile(join(folder, f)) and f.endswith(".txt")]
    return textfiles

In [330]:
f_files = list_files("Data/female")
m_files = list_files("Data/male")

X = f_files + m_files
y = ["female"] * len(f_files) + ["male"] * len(m_files)
print(len(X), len(y))

150 150


In [331]:
print("{0} Females \n{1} Males".format(len(f_files), len(m_files)))

58 Females 
92 Males


In [333]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

105 45
105 45


In [334]:
print("{0} Females".format(len([x for x in y_train if x == "female"])))
print("{0} Males".format (len([x for x in y_train if x == "male"]) ))

41 Females
64 Males


In [335]:
vectorizer = CountVectorizer(input="filename", analyzer="word", stop_words=) 

In [336]:
X_train_vectorized = vectorizer.fit_transform(X_train)

In [337]:
X_train_vectorized.shape

(105, 28330)

In [340]:
vectorizer.get_feature_names()[:500]

['00',
 '000',
 '03',
 '04',
 '043',
 '05',
 '06',
 '07',
 '09',
 '0907',
 '0oo',
 '10',
 '100',
 '1000',
 '100k',
 '100s',
 '101',
 '1017',
 '101on',
 '102',
 '106',
 '107',
 '10k',
 '10th',
 '10x',
 '11',
 '110',
 '112',
 '12',
 '120',
 '123',
 '12th',
 '12x',
 '13',
 '1312',
 '13th',
 '14',
 '1409618341378',
 '143',
 '1437',
 '14k',
 '15',
 '150',
 '1500',
 '151',
 '156',
 '15s',
 '15th',
 '15x',
 '16',
 '165th',
 '16th',
 '17',
 '170',
 '1738',
 '175',
 '1776',
 '1798430',
 '17th',
 '17x',
 '18',
 '180',
 '1800',
 '187',
 '187er',
 '19',
 '1953',
 '1957',
 '1959',
 '1965',
 '1968',
 '1970',
 '1975',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1986',
 '1988',
 '1989',
 '1991',
 '1992',
 '1993',
 '1995',
 '1998',
 '19__',
 '1da',
 '1raf7',
 '1st',
 '1x',
 '20',
 '200',
 '2000',
 '2001',
 '2003',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2014',
 '2016',
 '2073',
 '2084',
 '2093',
 '20s',
 '21',
 '210',
 '213',
 '21st',
 '22

In [341]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver="liblinear", random_state=0)

In [342]:
clf.fit(X_train_vectorized, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [343]:
X_test_vectorized = vectorizer.transform(X_test)
X_test_vectorized.shape

(45, 28330)

In [344]:
predictions = clf.predict(X_test_vectorized)

In [345]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy: ", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

Accuracy:  0.7777777777777778
             precision    recall  f1-score   support

     female       0.73      0.65      0.69        17
       male       0.80      0.86      0.83        28

avg / total       0.77      0.78      0.77        45

[[11  6]
 [ 4 24]]


In [346]:
pd.DataFrame(list(zip(X_test, y_test, predictions, y_test==predictions)), columns=["Instance", "Actual", "Predicted", "Correct"])

,Instance,Actual,Predicted,Correct
0,Data/male/Michael Jackson.txt,male,male,True
1,Data/male/Mr. Cheeks.txt,male,male,True
2,Data/female/Beyonce.txt,female,female,True
3,Data/female/Jody Watley.txt,female,female,True
4,Data/male/Levert.txt,male,female,False
5,Data/female/Phyllis Hyman.txt,female,female,True
6,Data/male/Jeremih.txt,male,male,True
7,Data/male/R. Kelly.txt,male,male,True
8,Data/female/Alessia Cara.txt,female,male,False
9,Data/male/George Michael.txt,male,female,False
